# Operating Systems & Concurrency: process API
_COSC 208, Introduction to Computer Systems, 2024-04-10_

## Announcements
* Project 2 revisions due Friday at 11pm
* Quiz 5 rescheduled for Friday, April 19

## Outline
* Warm-up: fork
* Waiting for processes
* Running a different program

## Warm-up: Limited Direct Execution (LDE)

* Q1: _One alternative to Limited Direct Execution (LDE) is **indirect execution**—i.e., for each assembly instruction an application wants to execute, the application must ask the OS to execute the instruction on the processor on the application’s behalf. Why is indirect execution a **poor design choice**?_

    Significant overhead

<div style="height:7em;"></div>

* Q2: _Another alternative to LDE is **direct execution**–i.e., an application can execute its assembly instructions directly on the processor and access any location in main memory. Why is direct execution a **poor design choice**?_

    An application can interfere with another application's memory; an application can use the processor indefinitely

<div style="height:7em;"></div>

* Q3: _What **processor feature(s)** make(s) LDE possible?_

    Two modes: kernel mode (processor is in this mode when the OS is running) and user mode (processor is in this mode when the application is running); `svc` instruction to make a system call; time interrupts 

<div style="height:7em;"></div>

* Q4: _What work (if any) must be done by an application, the operating system (OS), and the processor when an application wants to perform a privileged operation—e.g., start another application?_

    Application puts the system call number in a register and executes the `svc` instruction; Processor switches to kernel mode, saves the values in the registers on the OS's stack, and branches to the appropriate system call handling code based on the system call number; OS executes the system call; Processor restores the application's register values from the OS's stack, switches to user mode, and resumes execution of the application by executing the next instruction after the `svc` instruction

<div style="height:15em;"></div>

🛑 **STOP here** after completing the above questions; if you have extra time **take a few deep breaths** to the reduce stress.

<div style="page-break-after:always;"></div>

## Creating processes

* `int fork()`
    * Creates an exact copy of the running process, except for the return value from `fork` — return `0` to child (i.e., new) process; return child’s process ID to parent process (i.e., process that called fork)
    * Both child and parent resume execution from place where `fork` was called

* Q5: _What does the following code output?_

In [1]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  int main(int argc, char **argv) {
/* 4*/      int x = 13;
/* 5*/      printf("Before fork %d\n", x);
/* 6*/      fork();
/* 7*/      printf("After fork %d\n", x);
/* 8*/      return 0;
/* 9*/  }

Before fork 13
After fork 13
After fork 13


<div style="height:14em;"></div>

* Q6: _What does the following code output (assuming the new process has PID 1819)?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>    
/* 3*/  int main(int argc, char **argv) {
/* 4*/      printf("Before fork\n");
/* 5*/      int pid = fork();
/* 6*/      if (pid < 0) {
/* 7*/          printf("Fork failed\n");
/* 8*/          return 1;
/* 9*/      } else if (pid == 0) {
/*10*/          printf("Child gets %d\n", pid);
/*11*/      } else {
/*12*/          printf("Parent gets %d\n", pid);
/*13*/      }
/*14*/      return 0;
/*15*/  }

```
Before fork
Child gets 0
Parent gets 1819
```
OR
```
Before fork
Parent gets 1819
Child gets 0
``` 

<div style="height:15em;"></div>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

## Waiting for processes

* Wait for any child to finish — `int wait(int *status)`
    * Returns PID of the child process that finished
    * `status` parameter is optional
        * if passed a valid integer pointer, `wait` will store the return value of the child process's `main` function at the referenced memory location
        * if return value of child process's main function is not needed, then pass `NULL`
* Wait for a specific process to finish — `int waitpid(pid_t pid, int *status, int options)`
    * Returns PID of the process that finished
    * `pid` is PID if process to wait for — need not be a child process
    * `status` is the same as `wait`
    * `options` is typically `0`, except in special circumstances
* Wait functions do not return until child or specific process, respectively, finishes

* Q7: _What are all possible outputs of this program?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  int main() {
/* 4*/      int pid = fork();
/* 5*/      if (pid < 0) {
/* 6*/          printf("Fork failed\n");
/* 7*/          return 1;
/* 8*/      } else if (pid > 0) {
/* 9*/          wait(NULL);
/*10*/          printf("Parent\n");
/*11*/      } else {
/*12*/          printf("Child\n");
/*13*/      }
/*14*/      return 0;
/*15*/  }

_Postponed to next class_

<div style="height:17em;"></div>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

## Running a different program

* Example program

In [9]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  int main(int argc, char **argv) {
/* 4*/      printf("Begin\n");
/* 5*/      int pid = fork();
/* 6*/      if (pid < 0) {
/* 7*/          printf("Fork failed\n");
/* 8*/          return 1;
/* 9*/      } else if (pid == 0) {
/*10*/          printf("Child\n");
/*11*/          char *cmd[] = { "/bin/date", NULL };
/*12*/          execv(cmd[0], cmd);
/*13*/      } else {
/*14*/          printf("Parent\n");
/*15*/      }
/*16*/      printf("End\n");
/*17*/      return 0;
/*18*/  }

Begin
Parent
End
Child
Tue Apr  9 08:32:18 EDT 2024


## Extra practice

* Q8: _What does the following code output?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  int main() {
/* 4*/      int x = 10;
/* 5*/      int y = 20;
/* 6*/      int retval = fork();
/* 7*/      if (retval < 0) {
/* 8*/          printf("Fork failed\n");
/* 9*/          return 1;
/*10*/      } else if (retval > 0) {
/*11*/          y+= 5;
/*12*/      } else {
/*13*/          y -= 5;
/*14*/      }
/*15*/      printf("x=%d y=%d\n", x, y);
/*16*/      return 0;
/*17*/  }

```
x=10 y=25
x=10 y=15
```
OR
```
x=10 y=15
x=10 y=25
```

<div style="height:10em;"></div>

* Q9: _What are all possible outputs of this program (assuming the new process has PID 13346)?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  int main() {
/* 4*/      int pid = fork();
/* 5*/      printf("A %d\n", pid);
/* 6*/      if (pid == 0) {
/* 7*/          printf("B\n");
/* 8*/      } else if (pid > 0) {
/* 9*/          wait(NULL);
/*10*/          printf("C\n");
/*11*/      }
/*12*/  }

```
A 0
A 13346
B
C
```
OR
```
A 13346
A 0
B
C
```
OR
```
A 0
B
A 13346
C
```